In [293]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import warnings
import matplotlib.pyplot as plt
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet,Lasso, LassoCV, ElasticNetCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import seaborn as sns
# import xgboost as xgb

warnings.filterwarnings("ignore") 

%matplotlib inline

pd.options.display.max_columns = 10000  # None -> No Restrictions
pd.options.display.max_rows = 100    # None -> Be careful with this 
pd.options.display.max_colwidth = 100
pd.options.display.precision = 3

In [299]:
def dataCleaner(train, test):
    all_data = pd.concat((train,test))
    #log transform skewed numeric features:
    numeric_feats = all_data.dtypes[all_data.dtypes != "object"].index

    skewed_feats = train[numeric_feats].apply(lambda x: skew(x.dropna())) #compute skewness
    skewed_feats = skewed_feats[skewed_feats > 0.75]
    skewed_feats = skewed_feats.index  #Lets mark which features are very skewed.

    all_data[skewed_feats] = np.log1p(all_data[skewed_feats])
    #all_data['Estimated'] = np.log1p(all_data['Estimated'])

    all_data['quarter'] = all_data['quarter'].astype(object)
    all_data['year'] = all_data['year'].astype(object)
    
    #all_data.drop(['year'],axis=1,inplace=True)

    all_data = pd.get_dummies(all_data)
    all_data = all_data.fillna(0)
    
    X_train = all_data[:train.shape[0]]
    X_test = all_data[train.shape[0]:]
    y_train = X_train['revenue']
    y_test = X_test['revenue']
    X_train.drop(['revenue'],axis=1, inplace=True)
    X_test.drop(['revenue'],axis=1, inplace=True)
    
    X_train = X_train.reset_index(drop=True)
    y_train = y_train.reset_index(drop=True)
    X_test = X_test.reset_index(drop=True)
    y_test = y_test.reset_index(drop=True)
    return (X_train, y_train, X_test, y_test)

In [300]:
def rmse_cv(model, dataset, y):
    rmse= np.sqrt(-cross_val_score(model, dataset, y, scoring="neg_mean_squared_error", cv = 5))
    return(rmse)
    
def printRMSE_MSE(modelName,model,trainData,y):
    print(modelName +" has RMSE of "+ str(rmse_cv(model,trainData,y).mean()))
    yHat= model.predict(trainData)
    print(modelName + " has MSE on train data is: "+ str(mean_squared_error(y,yHat)))
    
def printTopTenFeatures(model, X_train):
    import operator
    coeff_used = np.sum(model.coef_!=0)
    print("Number of Coeffients used: "+ str(coeff_used))
    coef_dict = {} 

    indexOfFeature =0
    
#     switch (featureLabel) {
#         case 0:
#         case 1:
#         case 2:
#         case 3:
#         case 4:
#         case 5:
#         case :
    for coef in model.coef_:
        if coef!=0: 
            coef_dict[X_train.columns[indexOfFeature]]=coef
        indexOfFeature+=1
    highestCoef_dict_sorted = sorted(coef_dict.items(), key=operator.itemgetter(1), reverse=True)[:5]
    print("Highest Coef: ")
    print(highestCoef_dict_sorted)
    lowestCoef_dict_sorted = sorted(coef_dict.items(), key=operator.itemgetter(1), reverse=False)[:5]
    print("Lowest Coef: ")
    print(lowestCoef_dict_sorted)


In [301]:
full_quarter_df=pd.read_csv("southwest-airlines-analysis-full.csv")
full_quarter_wall_street_df=pd.read_csv("WallStreetEstimates.csv")
full_quarter_wall_street_df.drop(['Actual'],axis=1,inplace=True)
full_quarter_df = pd.merge(full_quarter_df,full_quarter_wall_street_df,on=['year','quarter'])
full_quarter_df.drop(['total_operating_expense','Estimated'],axis=1,inplace=True)
full_quarter_df

,citypair,aircraft,sum_departures_performed,sum_departures_scheduled,passengers,seats,avg_fuel_price,avg_stock_price,quarter,year,revenue,Last Quarter
0,RDU-MCO,Boeing 737-300,92.0,93.0,9139.0,12604.0,94.325,11.824,1,2013,4084,4200
1,RDU-MCO,Boeing 737-700/700LR/Max 7,162.0,164.0,16868.0,23100.0,94.325,11.824,1,2013,4084,4200
2,RDU-MDW,Boeing 737-700/700LR/Max 7,138.0,141.0,14527.0,19566.0,94.325,11.824,1,2013,4084,4200
3,RDU-PHL,Boeing 737-700/700LR/Max 7,120.0,122.0,11499.0,16986.0,94.325,11.824,1,2013,4084,4200
4,RDU-STL,Boeing 737-700/700LR/Max 7,82.0,83.0,6841.0,11678.0,94.325,11.824,1,2013,4084,4200
...,...,...,...,...,...,...,...,...,...,...,...,...
29515,TUL-STL,Boeing 737-700/700LR/Max 7,123.0,124.0,11647.0,17187.0,88.010,9.461,4,2012,4172,4300
29516,TUS-DEN,Boeing 737-300,47.0,48.0,5387.0,6439.0,88.010,9.461,4,2012,4172,4300
29517,TUS-MDW,Boeing 737-700/700LR/Max 7,120.0,121.0,14251.0,16992.0,88.010,9.461,4,2012,4172,4300
29518,TUS-SAN,Boeing 737-300,142.0,143.0,12916.0,19454.0,88.010,9.461,4,2012,4172,4300


In [302]:
def mod_stack(model, X_train, X_test, y_train, stack_num):
    for i in range(stack_num):
        model.fit(X_train,y_train)

        y_train_pred_np = model.predict(X_train)
        y_test_pred_np = model.predict(X_test)

        y_train_pred = pd.DataFrame(data=y_train_pred_np.flatten())
        y_test_pred = pd.DataFrame(data=y_test_pred_np.flatten())
        X_train['Stacker',i] = y_train_pred
        X_test['Stacker',i] = y_test_pred

In [ ]:
#TODO: Read in the monthly segment data
holistic_err_sum = 0
holistic_err_entries = 0
coeff_table = pd.DataFrame(columns=['Year/Quarter','Avg % Error','Wall Street % Error'])

for year in range(2013, 2020):
    for quarter in range(1,5):
        if(year == 2019 and (quarter == 3 or quarter == 4)):
            continue
        train = full_quarter_df
        test = full_quarter_df

        index_obj = train[(train['quarter'] == quarter) & (train['year'] == year)].index
        train = train.drop(index_obj)

        index_obj = test[(test['quarter'] != quarter) | (test['year'] != year)].index
        test = test.drop(index_obj)

        X_train, y_train, X_test, y_test = dataCleaner(train, test)
        # ridge regression
        modelForProblem1 = Ridge(alpha=2)
        mod_stack(modelForProblem1,X_train, X_test, y_train, 0)
        modelForProblem1.fit(X_train, y_train)
        
        y_train_pred_np = modelForProblem1.predict(X_train)
        y_test_pred_np = modelForProblem1.predict(X_test)
        
        y_train_pred = pd.DataFrame(data=y_train_pred_np.flatten())
        y_test_pred = pd.DataFrame(data=y_test_pred_np.flatten())

        y_train.index = range(len(y_train_pred))
        y_test.index = range(len(y_test_pred))
        y_train_pred = y_train_pred.join(y_train)
        y_test_pred = y_test_pred.join(y_test)
        y_train_pred.index = np.arange(1, len(y_train_pred) + 1)
        y_test_pred.index = np.arange(1, len(y_test_pred) + 1)

        y_train_pred.columns = ["Predicted","Expected"]
        y_test_pred.columns = ["Predicted","Expected"]
        y_train_pred["% Error"] = 100 *(abs(y_train_pred["Predicted"] - y_train_pred["Expected"]) / y_train_pred["Expected"])
        y_test_pred["% Error"] = 100 *(abs(y_test_pred["Predicted"] - y_test_pred["Expected"]) / y_test_pred["Expected"])
        print("Train Quarter:",quarter,"Year:",year,"Error %:",y_train_pred["% Error"].mean())
        print("Test Quarter:",quarter,"Year:",year,"Error %:",y_test_pred["% Error"].mean())

        #ws_perc_error = (100 *(abs(X_test["Estimated"].mean() - y_test_pred["Expected"].mean()) / y_test_pred["Expected"].mean()))

        holistic_err_sum += y_test_pred["% Error"].mean()
        holistic_err_entries = holistic_err_entries + 1

        df1 = pd.DataFrame({"Year/Quarter":[year+((quarter -1)*0.25)], "Avg % Error":y_test_pred["% Error"].mean(), "Wall Street % Error":ws_perc_error})
        coeff_table = coeff_table.append(df1)
        
        
        print("Model Prediction: ",y_test_pred["Predicted"].mean(),"Actual Value: ",y_test_pred["Expected"].min())
        #print("Wall street Estimate",X_test["Estimated"].mean())
print("Average Test % Error: ",holistic_err_sum / holistic_err_entries)

plt.plot('Year/Quarter','Avg % Error',data=coeff_table, marker='', linewidth=2, label='REG')
#plt.plot('Year/Quarter','Wall Street % Error',data=coeff_table, marker='', linewidth=2, label='WS')
plt.title('Southwest Revenue Estimation(stacked) contrasted with WS Model 2013-2019')
plt.xlabel('Years')
plt.ylabel('Average % Error for Revenue Estimate')
plt.legend()

Train Quarter: 1 Year: 2013 Error %: 1.113372178259698
Test Quarter: 1 Year: 2013 Error %: 2.5790525965843973
Model Prediction:  3978.671491955486 Actual Value:  4084
Train Quarter: 2 Year: 2013 Error %: 1.0046783984052576
Test Quarter: 2 Year: 2013 Error %: 4.328986558594652
Model Prediction:  4843.994845915551 Actual Value:  4643
Train Quarter: 3 Year: 2013 Error %: 1.130972797745223
Test Quarter: 3 Year: 2013 Error %: 0.9278837019620177
Model Prediction:  4502.836448803927 Actual Value:  4545
Train Quarter: 4 Year: 2013 Error %: 1.1327179033068473
Test Quarter: 4 Year: 2013 Error %: 0.7398676957571171
Model Prediction:  4394.270013258967 Actual Value:  4427
Train Quarter: 1 Year: 2014 Error %: 1.019967007923862
Test Quarter: 1 Year: 2014 Error %: 3.6067605778787373
Model Prediction:  4316.25764567443 Actual Value:  4166
Train Quarter: 2 Year: 2014 Error %: 1.0594039735332526
Test Quarter: 2 Year: 2014 Error %: 2.2860263284429765
Model Prediction:  4896.447220681717 Actual Value:  50

In [174]:
full_quarter_wall_street_df=pd.read_csv("WallStreetEstimates.csv")
100 *(abs(full_quarter_wall_street_df["Actual"] - full_quarter_wall_street_df["Estimated"]) / full_quarter_wall_street_df["Actual"])

0     0.177
1     0.508
2     0.194
3     0.351
4     0.358
5     0.174
6     1.215
7     0.380
8     0.380
9     0.174
10    0.615
11    1.373
12    1.373
13    0.558
14    0.208
15    0.000
16    3.947
17    0.784
18    0.227
19    0.217
20    0.208
21    1.796
22    0.000
23    0.227
24    0.440
25    0.431
26    0.488
27    0.476
dtype: float64

In [176]:
annual_int_rate = 0.039
qtr_int_rate = annual_int_rate / 4
full_quarter_wall_street_df['Basic Predictor Revenue'] = full_quarter_wall_street_df['Last Quarter'] * (1 + qtr_int_rate)
basic_pct_err = 100*(abs(full_quarter_wall_street_df['Actual'] - full_quarter_wall_street_df['Basic Predictor Revenue']))/full_quarter_wall_street_df['Actual']
basic_pct_err.mean()

6.998838687181768